In [25]:
import os

In [26]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant\\research'

In [27]:
os.chdir('../')

In [28]:
%pwd

'c:\\Users\\Appsb\\Desktop\\AI_Assistant'

# Data Ingestion

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    github_url: str
    save_dir: Path

In [30]:
from bot.constants import *
from bot.utils.common import read_yaml,create_directories

In [31]:
type(CONFIG_PATH)

pathlib._local.WindowsPath

In [33]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_PATH,
            params_path = PARAMS_PATH
        ):
        self.config = read_yaml(config_path)
        self.param = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            github_url=config.github_url,
            save_dir=config.save_dir
        )

        return data_ingestion_config

In [34]:
import os
import requests
from bot import logger
from dotenv import load_dotenv
load_dotenv()
import base64

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def get_repos(self):
        url = self.config.github_url
        headers = {"Authorization": f"token {os.getenv("TOKEN")}"}
        repos = requests.get(url, headers=headers).json()
        save_dir = self.config.save_dir
        with open(os.path.join(save_dir,'repo.txt'),'w') as f:
            logger.info(f"Starting writing data from git hub repos")
            for repo in repos:
                OWNER = repo['owner']['login']
                REPO = repo['name']
                readme = f"https://api.github.com/repos/{OWNER}/{REPO}/readme"
                headers = {"Authorization": f"token {os.getenv("TOKEN")}"}
                response = requests.get(readme, headers=headers)

                readme_content = "No Readme FIle"
                if response.status_code == 200:
                    data = response.json()
                    readme_content = base64.b64decode(data["content"]).decode("utf-8")
                
                f.write(str(repo["name"]) + " -> " + str(repo["html_url"] +"\n"))
                f.write("Description \n")
                f.write(readme_content)
                f.write(str("\n"))
                f.write(str("\n"))
                f.write("<=================================================================================================>")
                f.write(str("\n"))
                f.write(str("\n"))
            logger.info(f"Completed writing data from git hub repos")

In [54]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.get_repos()
except Exception as e:
    raise e

[2025-03-10 08:10:01,570: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-10 08:10:01,581: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-10 08:10:01,587: INFO: common: created directory at: artifacts]
[2025-03-10 08:10:01,590: INFO: common: created directory at: artifacts/data_ingestion]


[2025-03-10 08:10:02,072: INFO: 1434716802: Starting writing data from git hub repos]
[2025-03-10 08:10:15,028: INFO: 1434716802: Completed writing data from git hub repos]


In [ ]:
import requests
import os
TOKEN = os.getenv("TOKEN")
url = "https://api.github.com/user/repos?per_page=100"

headers = {"Authorization": f"token {TOKEN}"}
repos = requests.get(url, headers=headers).json()

repos[0]

In [ ]:
import requests
import base64

TOKEN = os.getenv("TOKEN")
OWNER = repos[0]['owner']['login']
REPO = repos[0]['name']#"DL-End_to_End"

readme = f"https://api.github.com/repos/{OWNER}/{REPO}/readme"
headers = {"Authorization": f"token {TOKEN}"}

response = requests.get(readme, headers=headers)

readme_content = "No Readme FIle"
if response.status_code == 200:
    data = response.json()
    readme_content = base64.b64decode(data["content"]).decode()
print(readme_content)  # Print or save to a file

No Readme FIle
